## Tensor Values

TensorFlow里面的数据单位就是Tensor(张量). 一个张量由任意维的数组值组成. rank表示张量的维数, shape是一个整数元祖(tuple), 表示每个维度上数组的长度. 例如

```python
3. # a rank 0 tensor; a scalar with shape [],
[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]
```

## TensorFlow Core

TensorFlow Core程序由两个独立的部分组成

1. 构建计算图(tf.Graph).
2. 运行计算图(tf.Session).

### Graph

计算图是一系列TensorFlow操作排列成的图. 该图由两种类型的对象组成. 
- Operations(或"ops")：图的节点. 描述消耗和产生张量的需要的计算.
- Tensor: 图中的边. 这些代表将通过图流动的值. 大多数TensorFlow函数返回`tf.Tensors`.

> 重要提示: tf.Tensors没有值, 它们只是计算图中元素的句柄.

In [ ]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # any {'0', '1', '2'} tensorflow suppress warnings

import numpy as np
import tensorflow as tf


a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

请注意, 打印张量不会像您所期望的那样输出值3.0,4.0和7.0. 以上仅构建计算图. 这些tf.Tensor对象仅代表这些操作将要运行的结果.

计算图中的每个操作都有一个唯一的名称. 这个名字与在Python中分配给对象的名称无关. 张量以产生它们的操作命名, 后跟输出索引, 如上面的`add：0`所示.

### TensorBoard

TensorFlow提供了一个名为TensorBoard的实用程序. TensorBoard的许多功能之一是可视化计算图. 您可以使用几个简单的命令轻松完成此操作.

1. 首先将计算图保存为TensorBoard摘要文件，如下所示:

In [ ]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

这将在当前目录中生成一个事件文件，其中包含以下形式的名称

```python
events.out.tfevents.{timestamp}.{hostname}
```

现在, 在一个新的终端中, 使用以下shell命令启动TensorBoard

```bash
tensorboard --logdir .
```

### Session

为了evaluate张量，实例化一个tf.Session对象，非正式地称为会话。会话封装了TensorFlow运行时的状态，并运行TensorFlow操作。如果`tf.Graph`像`.py`文件，则`tf.Session`就像`python`可执行文件。


下面的代码创建一个tf.Session对象，然后调用它的run方法来评估我们在上面创建的`total`张量：

In [ ]:
sess = tf.Session()
print(sess.run(total))

也可以传递多个Tensor, `run`方法可以流畅地处理元组或字典的任何组合，如下例所示：

In [ ]:
print(sess.run({'ab': (a, b), 'total': total}))

在调用`tf.Session.run`期间，任何tf.Tensor只有一个值。例如，下面的代码调用`tf.random_uniform`来产生一个tf.Tensor，它随机生成一个3元素向量（值范围为`[0,1)`）：

In [ ]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1, out2)))

**结果显示每次调用运行时都有一个不同的随机值**，但在单次运行期间（`out1`和`out2`得到的是相同的`vec`随机输入值）.

一些TensorFlow函数返回`tf.Operations`而不是`tf.Tensors`。在Operation上调用run的结果是None。运行一个操作是为了cause a side-effect，而不是检索一个值。这方面的例子包括initialization和稍后演示的training ops。

### Feeding

就目前来看，这张图并不特别有趣，因为它总是会产生一个常量的结果。图形可以被参数化以接受外部输入, 这种称为占位符的。占位符是稍后提供值，如函数参数。

In [ ]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

前面的三行有点像我们定义了一个有两个输入参数（x和y）的函数。我们可以通过使用run方法的feed_dict参数将多个输入的具体值提供给占位符来评估此图形。

In [ ]:
print(sess.run(z, feed_dict={x: 3, y: 4.5}))
print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

> 另请注意，feed_dict参数可用于覆盖graph中的任何张量。占位符和其他tf.Tensors之间唯一的区别是，如果没有给它们赋值，占位符会抛出错误。

### Datasets

占位符适用于简单的实验，但数据集是将数据流式传输到模型的首选方法.

要从Dataset中获取可运行的tf.Tensor，必须先将其转换为`tf.data.Iterator`，然后调用Iterator的`get_next`方法。

创建Iterator的最简单方法是使用`make_one_shot_iterator`方法。例如，在下面的代码中，`next_item`张量将在每次运行调用中从my_data数组返回一行：

In [ ]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

到达数据流的末尾会导致数据集抛出OutOfRangeError。例如，下面的代码读取next_item，直到没有更多数据要读取为止：

In [ ]:
while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break

## Layers

具有相同输入情况下, 可训练的模型必须修改graph中的值(参数)以获得新输出。Layers是将可训练参数添加到图形的首选方法。

Layers将变量和作用于它们的操作打包在一起。例如， densely-connected layer对每个输出的所有输入执行加权求和并应用可选激活函数。连接weights和biases由图层对象管理。

### Creating Layers

以下代码创建一个密集层，它接受一批输入向量，并为每个输出向量生成一个输出值。要将图层应用于输入，可以将该图层看成函数。例如：

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

图层检查其输入以确定其内部变量的大小。因此，在这里我们必须设置x占位符的形状，以便图层可以构建正确大小的权重矩阵.

既然我们已经定义了输出y的计算，那么在我们运行计算之前，还有一个细节需要处理。

### Initializing Layers

该图层包含了必须在可以使用之前进行初始化的变量。尽管可以单独初始化变量，但您可以轻松地初始化TensorFlow图中的所有变量，如下所示：

In [ ]:
init = tf.global_variables_initializer()
sess.run(init)

> 重要提示：调用`tf.global_variables_initializer`仅创建并返回TensorFlow操作的句柄。当我们用`tf.Session.run`运行它时，该操作将初始化所有全局变量。

另请注意，此`global_variables_initializer`仅在初始化程序创建时初始化图中存在的变量。所以初始化应该是图构建过程中添加的最后一件事情之一。

### Executing Layers

现在该图层已初始化，我们可以像处理其他任何张量一样评估linear_model的输出张量。例如，下面的代码：

In [ ]:
print(sess.run(y, {x: [[1, 2, 3],[4, 5, 6]]}))

### Layer Function shortcuts

对于每个图层类（如tf.layers.Dense），TensorFlow还提供了一个快捷方式功能（如tf.layers.dense）。唯一的区别是shortcut版本在一次调用中创建并运行图层。例如，以下代码等同于较早版本：

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y, {x: [[1, 2, 3], [4, 5, 6]]}))

虽然方便，但此方法不允许访问tf.layers.Layer对象。这使内省和调试更加困难，并且不能重用layers。

## Feature columns

实验特征列的最简单方法是使用`tf.feature_column.input_layer`函数。此函数只接受dense column为输入，因此要查看categorical column的结果，必须将其包含在`tf.feature_column.indicator_column`中。例如：

In [ ]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

运行`inputs`张量将把`feature`解析为一批向量.

特征列可以具有内部状态，比如图层，所以它们经常需要被初始化。Categorical columns在内部使用lookup表，这些需要单独的初始化操作`tf.tables_initializer`。

In [ ]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))

一旦内部状态被初始化，你可以像任何其他tf.Tensor一样运行`inputs`

In [ ]:
print(sess.run(inputs))

这显示了feature columns如何打包input vectors，其中前两个索引是one-hot "department" ，第三个是“sales”。


## Training

现在您已熟悉TensorFlow核心的基础知识，我们手动培训一个小型回归模型。

### Define the Data

首先让我们定义一些输入x和每个输入的预期输出y_true：

In [ ]:
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
y_true = tf.constant([[0], [-1], [-2], [-3]], dtype=tf.float32)

### Define the Model

接下来，建立一个简单的线性模型，带有1个输出：

In [ ]:
linear_model = tf.layers.Dense(units=1)

y_pred = linear_model(x)

您可以按如下方式计算预测结果

In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

该模型尚未训练，所以四个“预测”值不是很好。

### Loss

要优化模型，首先需要定义损失。我们将使用均方误差，这是回归问题的标准损失。

虽然您可以在较低级别的数学运算中手动执行此操作，但tf.losses模块提供了一组常用损失函数。您可以使用它来计算均方误差，如下所示：

In [ ]:
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print(sess.run(loss))

### Training

TensorFlow提供了执行标准优化算法的optimizer。这些实现为`tf.train.Optimizer`的子类。他们逐渐改变graph中每个变量以最小化损失。最简单的优化算法是梯度下降法，由`tf.train.GradientDescentOptimizer`实现。它根据相对于该变量的损失导数的大小来修改每个变量。例如：

In [ ]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

该代码构建了优化所需的所有graph组件，并返回一个训练操作。运行时，训练操作将更新图形中的变量。您可以按如下方式运行它：

In [ ]:
for i in range(10):
  _, loss_value = sess.run((train, loss))
  print(loss_value)
print(sess.run(y_pred))